# `quickmt` Benchmark

In [ ]:
from time import time

import datasets
from fire import Fire
from quickmt import M2m100Translator, NllbTranslator, OpusmtTranslator, Translator
from sacrebleu import BLEU, CHRF, TER

In [ ]:
bleu = BLEU()
chrf = CHRF()
ter = TER()

In [ ]:
src_lang = "it"
tgt_lang = "en"
src_lang_flores = "ita_Latn"
tgt_lang_flores = "eng_Latn"

quickmt_model_path = "../../../quickmt-models/quickmt-it-en"

opusmt_model_path = "../../../ct2-models/opus-mt-it-en-ct2"
opusmt_model_string = "Helsinki-NLP/opus-mt-it-en"

nllb600_model_path = "../../../ct2-models/nllb-200-distilled-600M-ct2"
nllb1b_model_path = "../../../ct2-models/nllb-200-distilled-1.3B-ct2"
m2m100_418m_model_path = "../../../ct2-models/m2m100_418-ct2/"
m2m100_1b_model_path = "../../../ct2-models/m2m100_1.2B-ct2/"

compute_device = "cpu"

In [ ]:
try:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{src_lang_flores}-{tgt_lang_flores}",
    )
except:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{tgt_lang_flores}-{src_lang_flores}",
    )

src = []
ref = []
for i in flores["devtest"]:
    src.append(i[f"sentence_{src_lang_flores}"])
    ref.append(i[f"sentence_{tgt_lang_flores}"])

In [ ]:
src[1]

In [ ]:
with open(f"flores.{src_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in src]))
with open(f"flores.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in ref]))

## Quickmt

In [ ]:
translator = Translator(model_path=quickmt_model_path, device=compute_device)

In [ ]:
src[1]

In [ ]:
translator(src[1], beam_size=5)

In [ ]:
translator(src[1], sampling_temperature=1.2, beam_size=1, sampling_topk=50, sampling_topp=0.9)

In [ ]:
%%time
mt = translator(src, max_batch_size=64)

In [ ]:
print(bleu.corpus_score(mt, [ref]))
print(chrf.corpus_score(mt, [ref]))
print(ter.corpus_score(mt, [ref]))

In [ ]:
with open(f"quickmt.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t quickmt.{tgt_lang} --batch_size 32 --only_system

## OpusMT

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model Helsinki-NLP/opus-mt-en-hi --output_dir ./ct2-models/opus-mt-en-hi-ct2
```


In [ ]:
translator = OpusmtTranslator(
    model_path=opusmt_model_path,
    model_string=opusmt_model_string,
    device=compute_device,
)

In [ ]:
translator(src[0])

In [ ]:
%%time
mt = translator(src)

In [ ]:
print(bleu.corpus_score(mt, [ref]))
print(chrf.corpus_score(mt, [ref]))
print(ter.corpus_score(mt, [ref]))

In [ ]:
with open(f"opusmt.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t opusmt.{tgt_lang} --batch_size 32 --only_system

## NLLB-600M

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/nllb-200-distilled-600M --output_dir ../ct2-models/nllb-200-distilled-600M-ct2
```

In [ ]:
translator = NllbTranslator(model_path=nllb600_model_path, device=compute_device)

In [ ]:
%%time
mt = translator(src, src_lang=src_lang_flores, tgt_lang=tgt_lang_flores)

In [ ]:
print(bleu.corpus_score(mt, [ref]))
print(chrf.corpus_score(mt, [ref]))
print(ter.corpus_score(mt, [ref]))

In [ ]:
with open(f"nllb-600m.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t nllb-600m.{tgt_lang} --batch_size 32 --only_system

## NLLB-1.3B

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/nllb-200-distilled-1.3B --output_dir ../ct2-models/nllb-200-distilled-1.3B-ct2
```

In [ ]:
translator = NllbTranslator(model_path=nllb1b_model_path, device=compute_device)

In [ ]:
%%time
mt = translator(src, src_lang=src_lang_flores, tgt_lang=tgt_lang_flores)

In [ ]:
print(bleu.corpus_score(mt, [ref]))
print(chrf.corpus_score(mt, [ref]))
print(ter.corpus_score(mt, [ref]))

In [ ]:
with open(f"nllb-1.3b.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
del mt

In [ ]:
! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t nllb-1.3b.{tgt_lang} --batch_size 16 --only_system

## M2M100-418M

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/m2m100_418M --output_dir ../ct2-models/m2m100_418-ct2/
ct2-transformers-converter --model facebook/m2m100_1.2B --output_dir ../ct2-models/m2m100_1.2B-ct2/
```

In [ ]:
translator = M2m100Translator(model_path=m2m100_418m_model_path, device=compute_device)

In [ ]:
%%time
mt = translator(src, src_lang=src_lang, tgt_lang=tgt_lang)

In [ ]:
print(bleu.corpus_score(mt, [ref]))
print(chrf.corpus_score(mt, [ref]))
print(ter.corpus_score(mt, [ref]))

In [ ]:
with open(f"m2m100-418m.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t m2m100-418m.{tgt_lang} --batch_size 32 --only_system

## M2M100-1.2B

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/m2m100_1.2B --output_dir ../ct2-models/m2m100_1.2B-ct2/
```

In [ ]:
translator = M2m100Translator(model_path=m2m100_1b_model_path, device=compute_device)

In [ ]:
%%time
mt = translator(src, src_lang=src_lang, tgt_lang=tgt_lang)

In [ ]:
print(bleu.corpus_score(mt, [ref]))
print(chrf.corpus_score(mt, [ref]))
print(ter.corpus_score(mt, [ref]))

In [ ]:
with open(f"m2m100-1.2B.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t m2m100-1.2B.{tgt_lang} --batch_size 32 --only_system

## Results

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-de-en            | 68.83 | 44.20   | 88.88   | 0.90     |
| Helsinki-NLP/opus-mt-de-en       | 66.16 | 40.04   | 87.68   | 3.51     |
| facebook/m2m100_418M             | 61.86 | 34.27   | 84.52   | 18.1     |
| facebook/m2m100_1.2B             | 65.99 | 40.34   | 87.67   | 35.2     |
| facebook/nllb-200-distilled-600M | 67.07 | 42.46   | 88.14   | 21.5     |
| facebook/nllb-200-distilled-1.3B | 68.75 | 44.44   | 89.08   | 37.4     |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-en-de            | 66.24 | 40.17   | 86.83   | 0.99     |
| Helsinki-NLP/opus-mt-en-de       | 63.53 | 36.06   | 84.63   | 3.66     |
| facebook/nllb-200-distilled-600M | 63.41 | 35.72   | 86.65   | 26.9     |
| facebook/nllb-200-distilled-1.3B | 65.01 | 38.61   | 87.99   | 46.0     |
| facebook/m2m100_418M             | 57.76 | 28.57   | 79.75   | 21.4     |
| facebook/m2m100_1.2B             | 63.37 | 36.24   | 85.82   | 41.0     |


| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-ko-en            | 36.96 | 14.97   | 87.10   | 1.25     |
| facebook/nllb-200-distilled-600M | 33.66 | 12.15   | 87.39   | 25.3     |
| facebook/nllb-200-distilled-1.3B | 35.62 | 13.23   | 88.39   | 40.2     |
| facebook/m2m100_418M             | 30.69 | 9.91    | 83.20   | 22.1     |
| facebook/m2m100_1.2B             | 33.26 | 11.35   | 85.65   | 41.0     |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-ko-en            | 56.25 | 27.03   | 86.11   | 0.91     |
| Helsinki-NLP/opus-mt-ko-en       | 50.39 | 20.78   | 83.06   | 3.81     |
| facebook/nllb-200-distilled-600M | 55.04 | 26.53   | 85.83   | 21.0     |
| facebook/nllb-200-distilled-1.3B | 57.56 | 29.61   | 87.24   | 37.2     |
| facebook/m2m100_418M             | 50.65 | 20.75   | 82.07   | 18.2     |
| facebook/m2m100_1.2B             | 62.35 | 24.59   | 85.15   | 34.7     |

| Model                            | chrf2 | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | -------- |
| quickmt/quickmt-en-ja            | 42.04 | 89.08   | 1.29     |
| Helsinki-NLP/opus-mt-en-jap      | 6.41  | 62.91   | 7.35     |
| facebook/nllb-200-distilled-600M | 30.00 | 86.64   | 28.4     |
| facebook/nllb-200-distilled-1.3B | 32.38 | 88.02   | 37.2     |
| facebook/m2m100_418M             | 32.73 | 85.09   | 24.7     |
| facebook/m2m100_1.2B             | 35.83 | 87.78   | 45.7     |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-it-en            |  |    |    |      |
| Helsinki-NLP/opus-mt-it-en       |  |    |    |      |
| facebook/nllb-200-distilled-600M | 61.97 | 33.49   | 87.39   | 23.0     |
| facebook/nllb-200-distilled-1.3B | 63.23 | 34.97   | 88.14   | 40.4     |
| facebook/m2m100_418M             | 56.94 | 25.92   | 83.14   | 20.0     |
| facebook/m2m100_1.2B             | 60.43 | 30.81   | 86.43   | 38.1     |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-it-en            | 61.48 | 32.10   | 87.31   | 0.9      |
| Helsinki-NLP/opus-mt-it-en       | 59.96 | 29.48   | 86.59   | 4.36     |
| facebook/nllb-200-distilled-600M | 61.97 | 33.49   | 87.39   | 23.0     |
| facebook/nllb-200-distilled-1.3B | 63.23 | 34.97   | 88.14   | 40.4     |
| facebook/m2m100_418M             | 56.94 | 25.92   | 83.14   | 20.0     |
| facebook/m2m100_1.2B             | 60.43 | 30.81   | 86.43   | 38.1     |


| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-en-it            |  |    |    |      |
| Helsinki-NLP/opus-mt-en-it       | 57.66 | 27.58   | 85.10   | 5.28     |
| facebook/nllb-200-distilled-600M | 58.30 | 28.58   | 87.30   | 25.2     |
| facebook/nllb-200-distilled-1.3B | 59.89 | 30.92   | 88.52   | 46.0     |
| facebook/m2m100_418M             | 54.34 | 23.91   | 82.64   | 21.4     |
| facebook/m2m100_1.2B             | 57.19 | 27.22   | 86.23   | 41.2     |